In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import re
from nltk import word_tokenize
from tashaphyne.normalize import normalize_searchtext
import joblib

import nltk
nltk.download('punkt')

app = Flask(__name__, template_folder='../Model Deployment with Flask')

stop_words =['من','في','على', 'و','فى','يا', 'عن','مع','ان','هو','علي','ما','اللي','كل',
 'بعد','ده','اليوم', 'أن','يوم','انا','إلى','كان','ايه','اللى','الى','دي','بين','انت','أنا','حتى','لما','فيه','هذا','واحد','احنا','اي','كده',
 'إن','او','أو','عليه','ف','دى','مين','الي','كانت','أمام','زي','يكون','خلال','ع','كنت','هي','فيها','عند','التي','الذي','قال','هذه','قد','انه','ريتويت','بعض','أول','ايه',
 'الان','أي','منذ','عليها','له','ال','تم','ب','دة','عليك'
 ,'اى','كلها','اللتى','هى','دا','انك','وهو', 'ومن','منك','نحن','زى','أنت','انهم','معانا','حتي','وانا','عنه','إلي','ونحن','وانت','منكم','وان','معاهم','معايا','وأنا','عنها','إنه',
 'اني','معك','اننا','فيهم','د','انتا','عنك','وهى','معا','آن','انتي','وأنت','وإن','ومع','وعن','معاكم','معاكو','معاها','وعليه','وانتم','وانتي','¿','|']

classes={0:'IQ', 1:'LY', 2:'QA', 3:'PL', 4:'SY', 5:'TN', 6:'JO', 7:'MA', 8:'SA', 9:'YE', 10:'DZ',
       11:'EG', 12:'LB', 13:'KW', 14:'OM', 15:'SD', 16:'AE', 17:'BH'}

@app.route('/')
def get_text():
    return render_template("index.html")

@app.route("/prediction", methods=["POST"])
def prediction():
    cvec = joblib.load('CountVectorizer.sav') 
    model = joblib.load('LogisticRegrassion_CV.sav')

    txt = request.form.get('text')
    text = re.sub("@\s[W+]\s",'', txt)    #remove @username
    text = re.sub("http\S+\s*",'', text)  #remove links http:------
    text = re.sub("[a-zA-Z0-9]",'', text)  #remove all non-arabic
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""),'', text) 
    text = re.sub("[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]+",
                                                  '', text) #remove emotions
    text = re.sub(r'(.)\1+', r'\1', text)  #remove repeating letters
    text = text.rstrip()

    original_words = []
    words = word_tokenize(text)
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    text = " ".join(original_words)

    text = normalize_searchtext(text)
    text_cvec= cvec.transform([text])
    predict = classes[model.predict(text_cvec)[0]]
    return render_template("index.html" ,data="Prediction : "+predict,txt="Text : "+txt)
    
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\original\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Mar/2022 20:42:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:42:22] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:42:34] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:42:57] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:43:10] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:43:22] "POST /prediction HTTP/1.1" 200 -
